In [ ]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Try all available connection strings in order until one works
candidates = [
    # Named Pipes (if enabled)
    r"DRIVER={ODBC Driver 17 for SQL Server};SERVER=np:\\localhost\pipe\MSSQL$SQLEXPRESS\sql\query;DATABASE=Funnel_Purchases;Trusted_Connection=yes;",
    # LocalDB (if database is on LocalDB)
    r"DRIVER={ODBC Driver 17 for SQL Server};SERVER=(localdb)\\MSSQLLocalDB;DATABASE=Funnel_Purchases;Trusted_Connection=yes;",
    # SQL Auth (replace with your credentials if you have them)
    r"DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost\\SQLEXPRESS;DATABASE=Funnel_Purchases;UID=sa;PWD=YourPassword;",
]

query = "SELECT TOP 10 * FROM dbo.[2019-Oct]"

for conn_str in candidates:
    try:
        print(f"Trying: {conn_str}")
        conn = pyodbc.connect(conn_str, timeout=5)
        df = pd.read_sql(query, conn)
        print("SUCCESS! Data preview:")
        print(df.head())
        conn.close()
        break
    except Exception as e:
        print(f"FAILED: {type(e)} {e}")
        continue
else:
    print("All connection attempts failed. Please check your SQL Server configuration and credentials.")


In [ ]:
# --- Helper: get a working DB connection from candidates ---
def get_conn(timeout=30):
    """Try connection strings defined in `candidates` and return an open connection."""
    for conn_str in candidates:
        try:
            print(f"Trying connection: {conn_str}")
            conn = pyodbc.connect(conn_str, timeout=timeout)
            print('Connection established')
            return conn
        except Exception as e:
            print(f"Connection failed: {type(e).__name__}: {e}")
            continue
    raise RuntimeError('All connection attempts failed - please check SQL Server access and credentials')

# Example usage: conn = get_conn(); df = pd.read_sql_query(SQL, conn); conn.close()

## Additional Analyses: Top products, Revenue by category, Users w/ >1 sessions, Funnel and Transitions


In [ ]:
# --- Top products (by event count and by purchases) ---
sql_top_products = "SELECT TOP 20 product_id, COUNT(*) AS cnt FROM dbo.[2019-Oct] GROUP BY product_id ORDER BY cnt DESC"
conn = None
try:
    conn = get_conn()
    top_products = pd.read_sql_query(sql_top_products, conn)
    print('Top products by event count (top 20):')
    print(top_products.head(10))
    if not top_products.empty:
        plt.figure(figsize=(12, 6))
        sns.barplot(x=top_products['product_id'].head(20), y=top_products['cnt'].head(20))
        plt.xticks(rotation=90)
        plt.title('Top 20 Products by Event Count')
        plt.xlabel('product_id')
        plt.ylabel('Event Count')
        plt.tight_layout()
        plt.show()
finally:
    if conn is not None:
        conn.close()

In [ ]:
# --- Revenue by category (purchases only) ---
sql_revenue_cat = "SELECT category_code, SUM(price) AS revenue, COUNT(*) AS purchases FROM dbo.[2019-Oct] WHERE event_type = 'purchase' GROUP BY category_code ORDER BY revenue DESC"
conn = None
try:
    conn = get_conn()
    revenue_cat = pd.read_sql_query(sql_revenue_cat, conn)
    print('Revenue by category (top 20):')
    print(revenue_cat.head(20))
    if not revenue_cat.empty:
        plt.figure(figsize=(12, 6))
        sns.barplot(x=revenue_cat['category_code'].fillna('UNKNOWN').head(20), y=revenue_cat['revenue'].head(20))
        plt.xticks(rotation=90)
        plt.title('Revenue by Category (purchases) - Top 20')
        plt.xlabel('Category')
        plt.ylabel('Revenue')
        plt.tight_layout()
        plt.show()
finally:
    if conn is not None:
        conn.close()

In [ ]:
# --- Users with more than 1 distinct sessions ---
sql_multi_sess = "SELECT user_id, COUNT(DISTINCT user_session) AS session_count FROM dbo.[2019-Oct] GROUP BY user_id HAVING COUNT(DISTINCT user_session) > 1 ORDER BY session_count DESC"
conn = None
try:
    conn = get_conn()
    users_multi = pd.read_sql_query(sql_multi_sess, conn)
    print('Users with >1 distinct sessions (top 20):')
    print(users_multi.head(20))
    # Quick visualization: distribution of session_count for these users
    if not users_multi.empty:
        plt.figure(figsize=(10, 5))
        # Focus on users with up to 10 sessions for a zoomed-in view
        max_sessions = min(10, users_multi['session_count'].max())
        sns.histplot(users_multi[users_multi['session_count'] <= max_sessions]['session_count'], bins=range(2, max_sessions+2), discrete=True)
        plt.title('Distribution of session counts for users with >1 sessions (up to 10 sessions)')
        plt.xlabel('Number of distinct sessions')
        plt.ylabel('Number of users')
        plt.xticks(range(2, max_sessions+1))
        plt.tight_layout()
        plt.show()
finally:
    if conn is not None:
        conn.close()

In [ ]:
# --- Funnel counts per session (view -> cart -> purchase) ---

# Updated logic: Only count sessions as 'purchased' if they also have a 'cart' event.
sql_funnel = """WITH session_flags AS (
  SELECT user_session,
    MAX(CASE WHEN event_type = 'view' THEN 1 ELSE 0 END) AS viewed,
    MAX(CASE WHEN event_type = 'cart' THEN 1 ELSE 0 END) AS carted,
    MAX(CASE WHEN event_type = 'purchase' THEN 1 ELSE 0 END) AS purchased
  FROM dbo.[2019-Oct]
  GROUP BY user_session
)
SELECT
  SUM(viewed) AS views,
  SUM(carted) AS carts,
  SUM(CASE WHEN carted = 1 AND purchased = 1 THEN 1 ELSE 0 END) AS purchases_with_cart
FROM session_flags"""

conn = None

try:

    conn = get_conn()

    funnel_df = pd.read_sql_query(sql_funnel, conn)

    print('Funnel session-level counts (updated logic):')

    print(funnel_df)

    if not funnel_df.empty:

        counts = funnel_df.iloc[0].to_dict()

        stages = ['Viewed', 'Carted', 'Purchased (with cart)']

        vals = [counts.get('views', 0), counts.get('carts', 0), counts.get('purchases_with_cart', 0)]

        total_sessions = vals[0] if vals[0] > 0 else sum(vals)

        pct = [v / total_sessions * 100 if total_sessions>0 else 0 for v in vals]

        plt.figure(figsize=(6, 4))

        sns.barplot(x=pct, y=stages, palette='Blues_d')

        for i, v in enumerate(pct):

            plt.text(v + 0.5, i, f"{int(vals[i])} ({v:.1f}%)", va='center')

        plt.xlabel('Percentage of sessions (base = sessions with view)')

        plt.title('Funnel: Viewed → Carted → Purchased (with cart)')

        plt.xlim(0, 100)

        plt.tight_layout()

        plt.show()

finally:

    if conn is not None:

        conn.close()

In [ ]:
# --- Transition counts (pairwise consecutive event transitions) and heatmap ---
sql_transitions = """WITH seq AS (
  SELECT user_session, event_time, event_type, ROW_NUMBER() OVER (PARTITION BY user_session ORDER BY event_time) AS rn
  FROM dbo.[2019-Oct]
),
pairs AS (
  SELECT s1.event_type AS from_type, s2.event_type AS to_type, COUNT(*) AS cnt
  FROM seq s1
  JOIN seq s2 ON s1.user_session = s2.user_session AND s1.rn + 1 = s2.rn
  GROUP BY s1.event_type, s2.event_type
)
SELECT * FROM pairs"""
conn = None
try:
    conn = get_conn()
    trans = pd.read_sql_query(sql_transitions, conn)
    print('Consecutive event transitions (sample):')
    print(trans.head(20))
    if not trans.empty:
        pivot = trans.pivot(index='from_type', columns='to_type', values='cnt').fillna(0)
        plt.figure(figsize=(8, 6))
        sns.heatmap(pivot, annot=True, fmt='.0f', cmap='YlGnBu')
        plt.title('Event transition counts (consecutive events within sessions)')
        plt.xlabel('To event')
        plt.ylabel('From event')
        plt.tight_layout()
        plt.show()
finally:
    if conn is not None:
        conn.close()

In [ ]:
# --- LOAD FULL DATA FROM SQL (SQL-only, no row limit) ---
full_query = "SELECT * FROM dbo.[2019-Oct]"  # adjust table as needed
loaded = False
for conn_str in candidates:
    try:
        print(f"Trying for full SQL load: {conn_str}")
        # increase timeout for large fetches
        conn = pyodbc.connect(conn_str, timeout=60)
        # Stream rows in chunks to avoid blocking the client while the server is sending a large result
        chunk_iter = pd.read_sql(full_query, conn, chunksize=100000)
        parts = []
        total = 0
        for chunk in chunk_iter:
            parts.append(chunk)
            total += len(chunk)
            print(f"Fetched {total} rows so far")
        if parts:
            df_sql = pd.concat(parts, ignore_index=True)
            print(f"Loaded {len(df_sql)} rows from SQL")
            # Replace preview df with full SQL data unconditionally (SQL-only mode)
            df = df_sql
        conn.close()
        loaded = True
        break
    except Exception as e:
        print(f"FAILED: {type(e)} {e}")
        continue
if not loaded:
    print('All SQL attempts failed for full load; keeping preview df (if any)')

# Trend Analysis


In [ ]:
# --- Event Distribution ---
event_counts = df['event_type'].value_counts()
print('Event Distribution:')
print(event_counts)

# --- Time Range ---
min_time = df['event_time'].min()
max_time = df['event_time'].max()
print(f'Event Time Range: {min_time} to {max_time}')

# --- Top Categories ---
if 'category_code' in df.columns:
    top_categories = df['category_code'].value_counts().head(10)
    print('Top 10 Categories:')
    print(top_categories)
else:
    print('No category_code column found.')

In [ ]:
# --- Plot: Event Distribution ---
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(8, 5))
sns.barplot(x=event_counts.index, y=event_counts.values)
plt.title('Event Distribution')
plt.xlabel('Event Type')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
# --- Plot: Time Range ---
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
plt.figure(figsize=(15, 1.5))
plt.hlines(1, min_time, max_time, colors='tab:blue', lw=6)
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.title('Event Time Range')
plt.xlabel('Date')
plt.yticks([])
plt.tight_layout()
plt.show()

In [ ]:
# --- Plot: Top 10 Categories ---
if 'category_code' in df.columns and not top_categories.empty:
    plt.figure(figsize=(10, 5))
    sns.barplot(x=top_categories.index, y=top_categories.values)
    plt.title('Top 10 Categories')
    plt.xlabel('Category Code')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print('No category_code data to plot.')

In [ ]:
# --- Plot: Daily Event Trends ---
# Summary info to explain single-date situation
print('Total rows:', len(df))
print('Unique dates in data:', df['event_time'].dt.date.nunique())
print('Unique sample dates:', df['event_time'].dt.date.unique()[:5])
# Ensure event_time is datetime and recompute daily counts reliably
df['event_time'] = pd.to_datetime(df['event_time'], errors='coerce')
# Use a DatetimeIndex (date only) to make plotting consistent
df['date'] = pd.to_datetime(df['event_time'].dt.date)
daily_counts = df.groupby('date').size().sort_index()
# Plot: handle single-day and multi-day cases so the plot is visible
if not daily_counts.empty:
    plt.figure(figsize=(35, 7))
    if len(daily_counts) == 1:
        # Single-day: bar plot with formatted x label and annotation to make count visible
        x = daily_counts.index.strftime('%Y-%m-%d')
        y = daily_counts.values
        ax = sns.barplot(x=x, y=y, color='tab:blue')
        plt.xlabel('Date')
        # annotate value on bar
        for p in ax.patches:
            height = p.get_height()
            ax.annotate(int(height), (p.get_x() + p.get_width() / 2., height),
                        ha='center', va='bottom', fontsize=12, color='black')
    else:
        # Multi-day: line with markers and annotations
        ax = sns.lineplot(x=daily_counts.index, y=daily_counts.values, marker='o', linewidth=2)
        plt.xlabel('Date')
        # annotate each point
        for x_val, y_val in zip(daily_counts.index, daily_counts.values):
            ax.annotate(str(int(y_val)), xy=(x_val, y_val), xytext=(0, 5), textcoords='offset points', ha='center')
    plt.title('Daily Event Trends')
    plt.ylabel('Number of Events')
    plt.xticks(rotation=45)
    plt.ylim(0, daily_counts.max() * 1.1)
    plt.tight_layout()
    plt.show()
else:
    print('No daily event data to plot. Check if event_time is within expected range or if data is loaded correctly.')

In [ ]:
# --- Debug: Check daily_counts and event_time ---
print('event_time dtype:', df['event_time'].dtype)
print('event_time min:', df['event_time'].min())
print('event_time max:', df['event_time'].max())
print('Sample event_time values:', df['event_time'].head())
print('daily_counts:')
print(daily_counts.head(20))

In [ ]:
# --- Plot: Hourly Event Trends ---
# Summary info to explain single-hour situation
print('Unique hours in data:', df['event_time'].dt.hour.nunique())
print('Unique sample hours:', df['event_time'].dt.hour.unique()[:10])
# Ensure event_time is datetime and compute hourly counts robustly
df['event_time'] = pd.to_datetime(df['event_time'], errors='coerce')
# Extract hour as integer (0-23) and compute counts
hourly_counts = df['event_time'].dt.hour.dropna().astype(int).value_counts().sort_index()
# Plot hourly counts; ensure x-axis shows 0-23 for context
if not hourly_counts.empty:
    plt.figure(figsize=(18, 6))
    ax = sns.lineplot(x=hourly_counts.index, y=hourly_counts.values, marker='o', linewidth=2)
    plt.title('Hourly Event Trends')
    plt.xlabel('Hour')
    plt.ylabel('Number of Events')
    # annotate points
    for x_val, y_val in zip(hourly_counts.index, hourly_counts.values):
        ax.annotate(str(int(y_val)), xy=(x_val, y_val), xytext=(0, 5), textcoords='offset points', ha='center')
    # Set xticks to full 0-23 range for clarity (it will show empty ticks if missing)
    plt.xticks(range(0, 24))
    plt.ylim(0, max(hourly_counts.max() * 1.1, 1))
    plt.tight_layout()
    plt.show()
else:
    print('No hourly event data to plot. Check if event_time is within expected range or if data is loaded correctly.')

In [ ]:
# Load data from SQL (assuming df is already loaded)
df['event_time'] = pd.to_datetime(df['event_time'])

# Assign step numbers within each session
df['step'] = df.groupby('user_session')['event_time'].rank(method='first').astype(int)

# Map funnel stages per session
funnel = df.groupby('user_session')['event_type'].agg(lambda x: set(x)).reset_index()
funnel['viewed'] = funnel['event_type'].apply(lambda x: 'view' in x)
funnel['carted'] = funnel['event_type'].apply(lambda x: 'cart' in x)
funnel['purchased'] = funnel['event_type'].apply(lambda x: 'purchase' in x)

# Calculate funnel counts
views = funnel['viewed'].sum()
carts = funnel['carted'].sum()
purchases_with_cart = ((funnel['carted']) & (funnel['purchased'])).sum()

# Visualize funnel
stages = ['Viewed', 'Carted', 'Purchased (with cart)']
vals = [views, carts, purchases_with_cart]
pct = [v / views * 100 if views > 0 else 0 for v in vals]

plt.figure(figsize=(6, 4))
sns.barplot(x=pct, y=stages, palette='Blues_d')
for i, v in enumerate(pct):
    plt.text(v + 0.5, i, f"{int(vals[i])} ({v:.1f}%)", va='center')
plt.xlabel('Percentage of sessions (base = sessions with view)')
plt.title('Funnel: Viewed → Carted → Purchased (with cart)')
plt.xlim(0, 100)
plt.tight_layout()
plt.show()